In [ ]:
!nvidia-smi

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import time
import sys
import matplotlib.pyplot as plt
import matplotlib.lines as lns
import numpy as np
import math


from google.colab import files
from functools import reduce

torch.manual_seed(1)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

dnload = True
batch_sz = 128

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=dnload, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_sz,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=dnload, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_sz,
                                         shuffle=False, num_workers=2)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):
    def __init__(self):
      super(Net, self).__init__()

    def forward(self, x):
      x = torch.flatten(x, 1)
      x = self.fc1(x)

      output = x
      return output


In [ ]:
for randseed in range(20):
      torch.manual_seed(10+randseed)

      net = Net()
      net.to(device)

      criterion = nn.CrossEntropyLoss()

      learning_rate = 0.3
      optimizer = optim.SGD(net.parameters(),  lr=0.3)


      train_losses = []
      val_losses = []
      num_epochs = 1500
      test_frequency= 1999
      batch_no=0
      net.train()

      for epoch in range(20):  # loop over the dataset multiple times
          start_time = time.time()
          running_loss = 0.0
          for i, data in enumerate(trainloader, 0):
              net.train()

              batch_no+=1

              # Update learning rate late in training
              if True:
                  learning_rate_disc = learning_rate/(np.sqrt(batch_no)/50)
              for param_group in optimizer.param_groups:
                  param_group['lr'] = learning_rate_disc

              inputs, labels = data[0].to(device), data[1].to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              c1=0.7
              c2=1.3

              if batch_no >=100:
                for param_group in optimizer.param_groups:
                  param_group['lr'] =torch.FloatTensor(1).uniform_(c1**(1/(batch_no%100+1)), c2**(1/(batch_no%100+1))).item() * learning_rate
    

              # forward + backward + optimize
              outputs = net(inputs)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              

              # print statistics
              running_loss += loss.item()
              if i % 200 ==  199:    # print every 200 mini-batches
                  print('[%d, %5d] loss: %.3f' %
                        (epoch + 1, i + 1, running_loss / 200))
                  train_losses.append(running_loss / 200)
                  running_loss = 0.0

                  losses = []
                  net.eval()
                  with torch.no_grad():
                      for j, data in enumerate(testloader):
                          startt_time = time.time()
                          inputs, labels = data[0].to(device), data[1].to(device)
                          outputs = net(inputs)
                          loss = criterion(outputs, labels)
                          losses.append(loss.item())
                  print('[%d, %5d] test loss: %.3f' %
                        (epoch + 1, i + 1,  np.mean((losses)) ))
                  val_losses.append(np.mean(losses))
                  net.train()


# save the results
output = ["{}{}".format("loss: ", "%.3f" % i) for i in train_losses]
output_conc = output
np.savetxt('MNIST_logreg_train'+str(10+randseed)+'.txt', output_conc, fmt="%s")
files.download('MNIST_logreg_train'+str(10+randseed)+'.txt')